In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, callbacks

2024-04-06 19:21:13.944263: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-06 19:21:13.944522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-06 19:21:14.063358: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-06 19:21:14.268260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 19:21:15.876033: W tensorflow/compiler/tf2

In [14]:
MODEL_NAME = "./treino/segundo_treino"
PASTA_FOURIER = "data/fourier/"
SPS = 200

In [123]:
def parse_data(example):
    parsed = tf.io.parse_sequence_example(example,  {
        "label": tf.io.FixedLenFeature(shape=6, dtype=tf.float32),
        "data" : tf.io.FixedLenFeature(shape=(20,100,2), dtype=tf.float32)
    })

    return parsed[0]["data"], parsed[0]["label"]

In [124]:
def get_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames).map(parse_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(32)

    return dataset

In [125]:
validate = get_dataset( PASTA_FOURIER + '0' ) 
train = get_dataset( [PASTA_FOURIER + '1', PASTA_FOURIER + '2', PASTA_FOURIER + '3'] ) 

In [144]:
for t in train.take(1):
    print(t[1].shape)

(32, 6)


In [146]:
# reg = tf.keras.regularizers.L1L2(l1=1, l2=1)
# reg = tf.keras.regularizers.l1()
# reg = tf.keras.regularizers.L2(l2=0.00000001)
reg=None
act = "relu"

model = models.Sequential()
model.add(layers.Conv2D(16, (2, 2), activation=act, kernel_regularizer=reg, input_shape=(20, 100, 2)))
model.add(layers.MaxPooling2D((2, 10)))
model.add(layers.Conv2D(32, (2, 2), activation=act, kernel_regularizer=reg))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (2, 2), activation=act, kernel_regularizer=reg))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(500, activation=act, kernel_regularizer=reg))
model.add(layers.Dense(60, activation=act, kernel_regularizer=reg))
model.add(layers.Dense(6, activation='softmax'))


In [147]:
# model=tf.keras.models.load_model(MODEL_NAME)

In [148]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 19, 99, 16)        144       
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 9, 9, 16)          0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 8, 8, 32)          2080      
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 4, 4, 32)          0         
 ng2D)                                                           
                                                                 
 conv2d_23 (Conv2D)          (None, 3, 3, 32)          4128      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 1, 1, 32)         

In [149]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy()
)

In [150]:
history = model.fit(
    train,
    epochs=1,
    # batch_size=10,
    # steps_per_epoch=2700,
    # shuffle=True,
    validation_data = validate,
    # validation_steps=100,
    verbose= 1,
    callbacks = [
         callbacks.EarlyStopping(monitor="val_loss", patience=2),
         callbacks.ModelCheckpoint(MODEL_NAME, save_best_only = True)
    ]
)


2024-04-06 20:11:35.935010: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-06 20:11:36.137368: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-06 20:11:36.896624: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-06 20:11:37.045231: I external/local_xla/xla/service/service.cc:168] XLA service 0x7d909f11a710 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-06 20:11:37.045277: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro M1000M, Compute Capability 5.0
2024-04-06 20:11:37.070345: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712427097.217970   10142 device_compiler.h:186] Com

   1444/Unknown - 12s 6ms/step - loss: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], color='red')

NameError: name 'history' is not defined